# Remove Rivers from Water Body Polygons

**What does this notebook do?** This notebook applies the `FilterRivers` filter from the `GenerateWaterBodyPolygons.ipynb` notebook. This allows this filtering step to be applied to the final Water Body Polygon dataset, in order to produce a further refined version in which the polygon IDs still match up with the all encompassing version.

**Required inputs:** 
* River line dataset for filtering out polygons comprised of river segments.
    * Variable name: `MajorRiversDataset`
    * Here we use the [Bureau of Meteorology's Geofabric v 3.0.5 Beta (Suface Hydrology Network)](ftp://ftp.bom.gov.au/anon/home/geofabric/), filtered to only keep features tagged as `major rivers`. 
    * There are some identified issues with this data layer that make the filtering using this data inconsistent (see [the discussion below](#rivers))
    * We therefore turn this off during the production of the water bodies shapefile.   

**Code history**
This notebook takes just the `FilterRivers` code from the `GenerateWaterBodyPolygons.ipynb` notebook.

**Date:** March 2019

**Author:** Claire Krause

**DEA module and environment version used**: `/dea-env/20181015`, `/dea/20181015` (See the accompanying [GenerateWaterBodyPolygonsRequirements.txt](files/GenerateWaterBodyPolygonsRequirements.txt) for a complete list of python libraries loaded into the environment).

In [1]:
import geopandas as gp

In [2]:
def Filter_shapefile_by_intersection(gpdData, gpdFilter, filtertype = 'intersects', invertMask = True, 
                                     returnInverse = False):
    '''
    Filter out polygons that intersect with another polygon shapefile. 
    
    Function copied from GenerateWaterBodyPolygons.ipynb.
    
    Parameters
    ----------
    
    gpdData: geopandas dataframe
        Polygon data that you wish to filter
    gpdFilter: geopandas dataframe
        Dataset you are using as a filter
    
    Optional
    --------
    filtertype: default = 'intersects'
        Options = ['intersects', 'contains', 'within']
    invertMask: boolean
        Default = 'True'. This determines whether you want areas that DO ( = 'False') or DON'T ( = 'True')
        intersect with the filter shapefile.
    returnInnverse: boolean
        Default = 'False'. If true, then return both parts of the intersection - those that intersect AND 
        those that don't as two dataframes.
    
    Returns
    -------
    gpdDataFiltered: geopandas dataframe
        Filtered polygon set, with polygons that intersect with gpdFilter removed.
    
    Optional
    --------
    if 'returnInverse = True'
    gpdDataFiltered, gpdDataInverse: two geopandas dataframes
        Filtered polygon set, with polygons that DON'T intersect with gpdFilter removed.
    '''    
    
    # Check that the coordinate reference systems of both dataframes are the same
    
    assert gpdData.crs == gpdFilter.crs, 'Make sure the the coordinate reference systems of the two provided dataframes are the same'
    
    Intersections = gp.sjoin(gpdFilter, gpdData, how="inner", op=filtertype)
    
    # Find the index of all the polygons that intersect with a river
    IntersectIndex = sorted(set(Intersections['index_right']))
    
    # Grab only the polygons NOT in the IntersectIndex
    # i.e. that don't intersect with a river
    if invertMask:
        gpdDataFiltered = gpdData.loc[~gpdData.index.isin(IntersectIndex)]
    else:
        gpdDataFiltered = gpdData.loc[gpdData.index.isin(IntersectIndex)]
    
    if returnInverse:
        # We need to use the indices from IntersectIndex to find the inverse dataset, so we
        # will just swap the '~'.
        
        if invertMask:
            gpdDataInverse = gpdData.loc[gpdData.index.isin(IntersectIndex)]
        else:
            gpdDataInverse = gpdData.loc[~gpdData.index.isin(IntersectIndex)]
            
        return gpdDataFiltered, gpdDataInverse
    else:    
        
        return gpdDataFiltered

## Load in the rivers dataset

We use the [Bureau of Meteorology's Geofabric v 3.0.5 Beta (Suface Hydrology Network)](ftp://ftp.bom.gov.au/anon/home/geofabric/) to filter out polygons that intersect with major rivers. This is done to remove river segments from the polygon dataset. We use the `SH_Network AHGFNetworkStream any` layer within the `SH_Network_GDB_V2_1_1.zip` geodatabase, and filter the dataset to only keep rivers tagged as `major`. It is this filtered dataset we use here.

Note that we reproject this dataset to `epsg 3577`, Australian Albers coordinate reference system. If this is not correct for your analysis, you can change this in the cell below.

### Note when using the Geofabric to filter out rivers

The option to filter out rivers was switched off for the production of our water bodies dataset. During testing, the Geofabric dataset was shown to lead to inconsistencies in what was removed, and what remained within the dataset. 

* The Geofabric continues the streamline through on-river dams, which means these polygons are filtered out. This may not be the desired result. 

![Stream and Dam intersection](OnRiverDam.JPG "An in-river dam that would be removed by the river filter, but may not be the desired result")


In [3]:
# Where is this file located?
MajorRiversDataset = '/g/data/r78/cek156/ShapeFiles/SH_Network_GDB_National_V3_0_5_Beta/SH_Network_GDB_National_V3_0_5_Beta_MajorFiltered.shp'

# Read in the major rivers dataset (if you are using it)
MajorRivers = gp.GeoDataFrame.from_file(MajorRiversDataset) 
MajorRivers = MajorRivers.to_crs({'init':'epsg:3577'})

## Read in the Water Bodies dataset

In [4]:
WaterPolygons = gp.read_file(f'/g/data/r78/cek156/dea-notebooks/WaterbodyAreaMappingandMonitoring/MDBANSWAllTime01-005HybridWaterbodies/NSWMDBWaterBodies.shp')
WaterPolygons = WaterPolygons.to_crs({'init':'epsg:3577'})

## Filter out polygons that intersect with a major river

In [5]:
WaterBodiesBigRiverFiltered = Filter_shapefile_by_intersection(WaterPolygons, MajorRivers)

## Write out the amended shapefile

In [6]:
WaterBodiesBigRiverFiltered.crs = {'init': 'epsg:3577'}
WaterBodiesBigRiverFiltered.to_file('/g/data/r78/cek156/dea-notebooks/WaterbodyAreaMappingandMonitoring/MDBANSWAllTime01-005HybridWaterbodies/NSWMDBWaterBodiesRiverFiltered.shp', driver='ESRI Shapefile')